<a href="https://colab.research.google.com/github/yeeh-l/Signal-System-Hw2020/blob/circleci-project-setup/ch07_Ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 >Ch07-Exercises            資工二_B0729066_葉冠志<br></h1>


In [4]:
# get thinkdsp 

!wget "https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkdsp.py"
!wget "https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkplot.py"

#get wav

!wget "https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/92002__jcveliz__violin-origional.wav"
!mkdir wav
!cp 92002__jcveliz__violin-origional.wav wav/
import thinkdsp 
import thinkplot
import numpy as np
wavFilename= '/content/92002__jcveliz__violin-origional.wav'

--2020-06-09 05:39:21--  https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkdsp.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48703 (48K) [text/plain]
Saving to: ‘thinkdsp.py.2’

thinkdsp.py.2       100%[===================>]  47.56K  --.-KB/s    in 0.03s   

2020-06-09 05:39:21 (1.84 MB/s) - ‘thinkdsp.py.2’ saved [48703/48703]

--2020-06-09 05:39:23--  https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkplot.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22756 (22K) [text/plain]
Saving to: 

Exercise 2 is very interesting, study it and do a report¶
Exercise 2
In this chapter, I showed how we can express the DFT and inverse DFT as matrix multiplications. These operations take time proportional to $N ^2$, where N is the length of the wave array. That is fast enough for many applications, but there is a faster algorithm, the Fast Fourier Transform (FFT), which takes time proportional to N logN. The key to the FFT is the Danielson-Lanczos lemma:

$
DFT(y)[n] = DFT(e)[n] + exp(−2 π i n / N) DFT(o)[n] 
$

Where,

$DFT(y)[n]$ is the $n^{th}$ element of the DFT of y;
e is a wave array containing the even elements of y,
and o contains the odd elements of y.
This lemma suggests a recursive algorithm for the DFT:

Given a wave array, y, split it into its even elements, e, and its odd elements, o.
Compute the DFT of e and o by making recursive calls.
Compute DFT(y) for each value of n using the Danielson-Lanczos lemma.
For the base case of this recursion, you could wait until the length of y is 1. In that case, DFT(y) = y. Or if the length of y is sufficiently small, you could compute its DFT by matrix multiplication, possibly using a precomputed matrix.

Hint:

I suggest you implement this algorithm incrementally by starting with a version that is not truly recursive. In Step 2, instead of making a recursive call, use dft, as defined by Section 7.7, or np.fft.fft. Get Step 3 working, and confirm that the results are consistent with the other implementations. Then add a base case and confirm that it works. Finally, replace Step 2 with recursive calls.

One more hint: Remember that the DFT is periodic; you might find np.tile useful.

You can read more about the FFT at https://en.wikipedia.org/wiki/Fast_Fourier_transform.

<HR><H3>

作為測試案例，我將從一個小的實信號開始，計算它的FFT。

In [0]:
PI2 = 2*np.pi

In [7]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


<H3>
這是我對書中DFT的實現。

In [0]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

<H3>我們可以確認，這個實現得到的結果是一樣的。

In [31]:
hs2 = dft(ys)
print(sum(abs(hs - hs2)))

5.864775846765962e-16


<H3>作為製作遞歸FFT的一個步驟，我將從一個分割輸入數組並使用np.fft.fft計算半數的FFT的版本開始。

In [0]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

<H3>然後我們得到相同結果:

In [33]:
hs3 = fft_norec(ys)
print(sum(abs(hs - hs3)))

0.0


<H3>
最後，我們可以用遞歸調用代替np.fft.fft，並增加一個基例。

In [0]:

def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [35]:
hs4 = fft(ys)
print(sum(abs(hs - hs4)))

1.6653345369377348e-16



<H3>這種FFT的實現需要的時間與$n \log n$成正比。它所花費的空間也與$n log n$成正比，而且它浪費了一些製作和復制數組的時間。它可以被改進為 "原地 "運行；在這種情況下，它不需要額外的空間，並且在開銷上花費的時間更少。

<H1>END<BR><H3>ch07-Ex2 HW 資工二_B0729066_葉冠志
<HR>